# ANN to model Cl

In [1]:
# Create dataset
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import torch

results_dir = "results/"
cuda0 = torch.device("cuda:0")

class AerofoilLiftSample:
    def __init__(self, file_name):
        self.file_name = file_name
        aerofoil_data = self.file_name.split("_")
        naca = aerofoil_data[1]
        self.camber = naca[0]
        self.camber_pos = naca[1]
        self.thickness = naca[2:3]
        self.reynolds = aerofoil_data[2]
        self.mach = aerofoil_data[3]
        self.alpha = aerofoil_data[4].split(".")[0]

    def __str__(self):
        return self.file_name

    def tensor(self):
        return torch.tensor([float(self.camber),
                             float(self.camber_pos),
                             float(self.thickness),
                             float(self.reynolds),
                             float(self.mach),
                             float(self.alpha),
                             ],
                            dtype=torch.float, device=cuda0)

    def cp_data(self):
        file = results_dir + self.file_name
        df = pd.read_csv(file, sep="\s+", skiprows=[0,1,2], names=["x", "y", "Cp"], dtype=float, error_bad_lines=False)
        return df

    def cl(self):
        return torch.tensor(np.trapz(self.cp_data()["Cp"], self.cp_data()["x"]), dtype=torch.float, device=cuda0)

class AerofoilLiftDataset(Dataset):
    """Aerofoil Coefficient of Lift Dataset"""
    def __init__(self):
        self.samples = self.load_samples()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, item):
        return self.samples[item].tensor(), self.samples[item].cl()

    def load_samples(self):
        samples = []
        for file in os.listdir(results_dir):
            if "cpwr" in file:
                samples.append(AerofoilLiftSample(file))
        return samples

In [13]:
from torch.utils.data import DataLoader, random_split

dataset = AerofoilLiftDataset()
print(dataset[0])

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataLoader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dataLoader = DataLoader(test_dataset, batch_size=8)

(tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+08,  1.5000e-01,
        -2.0000e+00], device='cuda:0'), tensor(-0.2261, device='cuda:0'))


C:\Users\Sam\AppData\Local\Temp/ipykernel_11284/1112131231.py:42: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  return torch.tensor(np.trapz(self.cp_data()["Cp"], self.cp_data()["x"]), dtype=torch.float, device=cuda0)


In [39]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

model = torch.nn.Sequential(
    torch.nn.Linear(6, 4096),
    torch.nn.ReLU(),
    torch.nn.Linear(4096, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 32),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 1),
)
model.to(cuda0)
loss_fn = torch.nn.L1Loss()
optim = torch.optim.Adam(model.parameters(), lr =0.001)


for epoch in range(500):
    i=0
    print(f"Epoch:{epoch}")
    L = []
    for train_features, train_labels in train_dataLoader:
        i+=1
        train_labels = torch.reshape(train_labels, (-1,1))
        #break
        x = train_features
        y_pred = model(x)
        loss = loss_fn(y_pred, train_labels)
        optim.zero_grad()
        loss.backward()
        optim.step()
        L.append(loss.item())
        writer.add_scalar('tl', loss.item(),i)
        #print(loss.item())

    writer.add_scalar('epoch training loss', np.nanmean(L),epoch)


Epoch:0


C:\Users\Sam\AppData\Local\Temp/ipykernel_11284/1112131231.py:42: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  return torch.tensor(np.trapz(self.cp_data()["Cp"], self.cp_data()["x"]), dtype=torch.float, device=cuda0)


Epoch:1
Epoch:2
Epoch:3
Epoch:4
Epoch:5
Epoch:6
Epoch:7
Epoch:8
Epoch:9
Epoch:10
Epoch:11
Epoch:12
Epoch:13
Epoch:14
Epoch:15
Epoch:16
Epoch:17
Epoch:18
Epoch:19
Epoch:20
Epoch:21
Epoch:22
Epoch:23
Epoch:24
Epoch:25
Epoch:26
Epoch:27
Epoch:28
Epoch:29
Epoch:30
Epoch:31
Epoch:32
Epoch:33
Epoch:34
Epoch:35
Epoch:36
Epoch:37
Epoch:38
Epoch:39
Epoch:40
Epoch:41
Epoch:42
Epoch:43
Epoch:44
Epoch:45
Epoch:46
Epoch:47
Epoch:48
Epoch:49
Epoch:50
Epoch:51
Epoch:52
Epoch:53
Epoch:54
Epoch:55
Epoch:56
Epoch:57
Epoch:58
Epoch:59
Epoch:60
Epoch:61
Epoch:62
Epoch:63
Epoch:64
Epoch:65
Epoch:66
Epoch:67
Epoch:68
Epoch:69
Epoch:70
Epoch:71
Epoch:72
Epoch:73
Epoch:74
Epoch:75
Epoch:76
Epoch:77
Epoch:78
Epoch:79
Epoch:80
Epoch:81
Epoch:82
Epoch:83
Epoch:84
Epoch:85
Epoch:86
Epoch:87
Epoch:88
Epoch:89
Epoch:90
Epoch:91
Epoch:92
Epoch:93
Epoch:94
Epoch:95
Epoch:96
Epoch:97
Epoch:98
Epoch:99
Epoch:100
Epoch:101
Epoch:102
Epoch:103
Epoch:104
Epoch:105
Epoch:106
Epoch:107
Epoch:108
Epoch:109
Epoch:110
Epoch:11

KeyboardInterrupt: 

In [41]:
print(model(torch.tensor([2,4,14, 1e8, 0.4,3], device=cuda0)))

tensor([0.6461], device='cuda:0', grad_fn=<AddBackward0>)
